In [1]:
import pandas as pd
import numpy as np
from q2_extractor import Extractor
from qiime2 import Artifact

## Input Data
Change the `alpha_depth` parameter and re-run the code to get the results for a different read depth.

Higher read depth excludes more samples.

Default for GitHub push is set to 1000.

In [2]:
metadata = pd.read_csv("data/METADATA.txt", sep="\t", index_col=0)
tax = Artifact.load("artifacts/taxonomy.qza")
asv_table = Artifact.load("artifacts/unfiltered_table.qza")
md = metadata
alpha_depths = [500, 1000, 2000, 4000, 6000, 8000, 10000]
alpha_depth = 1000
#25 replicates were computed for the alpha diversity values
for i in range(1,26):
    alpha_pd = Artifact.load("artifacts/diversity_" + str(alpha_depth) + "_" + str(i) + "/faith_pd_vector.qza")
    alpha_asvs = Artifact.load("artifacts/diversity_" + str(alpha_depth) + "_" + str(i) + "/observed_otus_vector.qza")
    alpha_shannon = Artifact.load("artifacts/diversity_" + str(alpha_depth) + "_" + str(i) + "/shannon_vector.qza")
    phyd = alpha_pd.view(pd.Series)
    phyd.name = phyd.name + "_" + str(i)
    asvs = alpha_asvs.view(pd.Series)
    asvs.name = asvs.name + "_" + str(i)
    shannon = alpha_shannon.view(pd.Series)
    shannon.name = shannon.name + "_" + str(i)
    md = md.merge(phyd, left_index=True, right_index=True)
    md = md.merge(asvs, left_index=True, right_index=True)
    md = md.merge(shannon, left_index=True, right_index=True)

In [3]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

In [4]:
def alpha_boxplot(metadata_category, alpha_measure):
    data = []
    for val in np.unique(md[metadata_category]):
        data.append(go.Box(boxpoints='all',
                    name=val,
                    y=md[[alpha_measure + "_" + str(i) for i in range(1,26)]].where(md[metadata_category]==val).dropna().mean(axis=1))
                    )
    plotly.offline.iplot(go.Figure(data=data, layout=go.Layout(title=alpha_measure)))

In [5]:
alpha_boxplot('mode_of_delivery', 'faith_pd')
alpha_boxplot('mode_of_delivery', 'shannon')
alpha_boxplot('mode_of_delivery', 'observed_otus')

In [6]:
alpha_boxplot('blood_group', 'faith_pd')
alpha_boxplot('blood_group', 'shannon')
alpha_boxplot('blood_group', 'observed_otus')